In [1]:
import os
import json
import requests
import pandas as pd
from itertools import combinations 
import pulp 
from icecream import ic
import numpy as np
import json

In [2]:
def load_API_Key():    
    api_key = '' #todo: create env variable
    return api_key

# Global Variables

In [4]:
api_key = load_API_Key()
date = "2024-09-06"
season = '2024'
week = 1
game_has_been_played = True
projection_data_has_been_downloaded = True
stats_data_has_been_downloaded = True

# Download the DFS slate for a given date and save it to CSV

In [6]:
def download_slate_json(api_key, date):
    
    url = f"https://api.sportsdata.io/v3/nfl/projections/json/DfsSlatesByDate/{date}?key={api_key}"
    response = requests.get(url)  # Make the HTTP GET request
    if response.status_code == 200:
        try:
            json_data = response.json()
            if len(json_data) > 0:
                json_string = json.dumps(json_data, indent=4)
                directory = f'sportsdata/DFS/{date}/'
                os.makedirs(directory, exist_ok=True)
                json_file = f'{directory}Slate.json'
                with open(json_file, 'w') as file:
                    file.write(json_string)
                ic(json_file)      
        except ValueError:
            print("Response is not in JSON format.")
            print(response.text)  # Print the response text for debugging
    else:
        print(f"Error: {response.status_code}")
        print(response.text) 
    

In [7]:
def download_projection_json(api_key, date, season, week):
    
    url = f"https://api.sportsdata.io/v3/nfl/projections/json/PlayerGameProjectionStatsByWeek/{season}/{week}?key={api_key}"
    response = requests.get(url)  # Make the HTTP GET request
    if response.status_code == 200:
        try:
            json_data = response.json()
            if len(json_data) > 0:
                json_string = json.dumps(json_data, indent=4)
                directory = f'sportsdata/DFS/{date}/'
                os.makedirs(directory, exist_ok=True)
                json_file = f'{directory}Projections.json'
                with open(json_file, 'w') as file:
                    file.write(json_string)
                ic(json_file)      
        except ValueError:
            print("Response is not in JSON format.")
            print(response.text)  # Print the response text for debugging
    else:
        print(f"Error: {response.status_code}")
        print(response.text) 

In [8]:
def download_defense_projection_json(api_key, date, season, week):
    
    url = f"https://api.sportsdata.io/v3/nfl/projections/json/FantasyDefenseProjectionsByGame/{season}/{week}?key={api_key}"
    response = requests.get(url)  # Make the HTTP GET request
    if response.status_code == 200:
        try:
            json_data = response.json()
            if len(json_data) > 0:
                json_string = json.dumps(json_data, indent=4)
                directory = f'sportsdata/DFS/{date}/'
                os.makedirs(directory, exist_ok=True)
                json_file = f'{directory}DefenseProjections.json'
                with open(json_file, 'w') as file:
                    file.write(json_string)
                ic(json_file)      
        except ValueError:
            print("Response is not in JSON format.")
            print(response.text)  # Print the response text for debugging
    else:
        print(f"Error: {response.status_code}")
        print(response.text) 

In [9]:
def download_stats_json(api_key, date, season, week, team, endpoint):
    
    url = f"https://api.sportsdata.io/v3/nfl/stats/json/{endpoint}/{season}/{week}/{team}?key={api_key}"
    response = requests.get(url)  # Make the HTTP GET request
    if response.status_code == 200:
        try:
            json_data = response.json()
            if len(json_data) > 0:
                json_string = json.dumps(json_data, indent=4)
                directory = f'sportsdata/DFS/{date}/'
                os.makedirs(directory, exist_ok=True)
                json_file = f'{directory}{date}/{team}-def-stats.json'
                with open(json_file, 'w') as file:
                    file.write(json_string)
                ic(json_file)      
        except ValueError:
            print("Response is not in JSON format.")
            print(response.text)  # Print the response text for debugging
    else:
        print(f"Error: {response.status_code}")
        print(response.text) 

# Check if projection_data_has_been_downloaded has been initialized

In [11]:
if not projection_data_has_been_downloaded:
    download_slate_json(api_key, date) #To get Slate
    download_projection_json(api_key, date, season, week) #To get projections
    download_defense_projection_json(api_key, date, season, week) #To get defensive projections

# Read json and parse into dataframes

In [13]:
def read_slate_json_file(date):
    data_path = f'sportsdata'
    json_file = f'{data_path}/DFS/{date}/Slate.json'
    # Open and read the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)
    df = pd.read_json(json_file)
    df_roster_slots = pd.json_normalize(data, record_path=['SlateRosterSlots'])
    df_players = pd.json_normalize(data, record_path=['DfsSlatePlayers'])
    df_games = pd.json_normalize(data, record_path=['DfsSlateGames'])
    return df, df_players, df_games, df_roster_slots

In [14]:
def read_projections_json_file(date):
    data_path = f'sportsdata'
    json_file = f'{data_path}/DFS/{date}/Projections.json'
    # Open and read the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)
    df = pd.read_json(json_file)
    return df

In [15]:
def read_defense_projections_json_file(date):
    data_path = f'sportsdata'
    json_file = f'{data_path}/DFS/{date}/DefenseProjections.json'
    # Open and read the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)
    df = pd.read_json(json_file)
    return df

In [16]:
def read_stats_json_file(date, team):
    data_path = f'sportsdata'
    json_file = f'{data_path}/DFS/{date}/{team}-stats.json'
    # Open and read the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)
    df = pd.read_json(json_file)
    return df

In [17]:
def read_def_stats_json_file(date, team):
    data_path = f'sportsdata'
    json_file = f'{data_path}/DFS/{date}/{team}-def-stats.json'
    # Open and read the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)
    df = pd.read_json(json_file)
    return df

In [18]:
df, df_players, df_games, df_roster_slots = read_slate_json_file(date)

In [19]:
df.head()

,SlateID,Operator,OperatorSlateID,OperatorName,OperatorDay,OperatorStartTime,NumberOfGames,IsMultiDaySlate,RemovedByOperator,OperatorGameType,SalaryCap,SlateRosterSlots,DfsSlateGames,DfsSlatePlayers
0,12156,DraftKings,112696,GB vs PHI,2024-09-06T00:00:00,2024-09-06T20:15:00,1,False,False,Showdown Captain Mode,50000,"[CPT, FLEX, FLEX, FLEX, FLEX, FLEX]","[{'SlateGameID': 44096, 'SlateID': 12156, 'Gam...","[{'SlatePlayerID': 2396406, 'SlateID': 12156, ..."


In [20]:
operator_name = df['OperatorName'].loc[0]
print(operator_name)

GB vs PHI


# Check if game_has_been_played and stats_data_has_been_downloaded has been initialized

In [53]:
teams = operator_name.split(' vs ')

# Resulting strings
team1 = teams[0] 
team2 = teams[1] 
endpoint = 'PlayerGameStatsByTeamFinal'
defense_endpoint = 'FantasyDefenseByGameByTeam'
if game_has_been_played and not stats_data_has_been_downloaded:
    download_stats_json(api_key, date, season, week, team1, endpoint)
    download_stats_json(api_key, date, season, week, team2, endpoint)
    download_stats_json(api_key, date, season, week, team1, defense_endpoint)
    download_stats_json(api_key, date, season, week, team2, defense_endpoint)


In [55]:
if game_has_been_played:
    team1_stats_df = read_stats_json_file(date, team1)
    team2_stats_df = read_stats_json_file(date, team2)
    team1_stats_df.head()
    team1_def_stats_df = read_def_stats_json_file(date, team1)
    team2_def_stats_df = read_def_stats_json_file(date, team2)

In [57]:
print(df_roster_slots[0].tolist())

['CPT', 'FLEX', 'FLEX', 'FLEX', 'FLEX', 'FLEX']


In [59]:
df_players['ProjFantasyPoints'] = 0
df_players['ActualFantasyPoints'] = 0
print(df_players.head())

   SlatePlayerID  SlateID  SlateGameID  PlayerID  PlayerGameProjectionStatID  \
0        2396406    12156      44096.0     21227                 991161746.0   
1        2396407    12156      44096.0     21227                 991161746.0   
2        2396408    12156      44096.0     21797                 991161770.0   
3        2396409    12156      44096.0     21797                 991161770.0   
4        2396410    12156      44096.0     24725                 991161904.0   

   FantasyDefenseProjectionStatID OperatorPlayerID OperatorSlatePlayerID  \
0                             NaN           824428              35703473   
1                             NaN           824428              35703478   
2                             NaN           922017              35703472   
3                             NaN           922017              35703477   
4                             NaN          1049552              35703474   

  OperatorPlayerName OperatorPosition  OperatorSalary Team  Te

# Read Projections json into dataframe

In [62]:
projections_df = read_projections_json_file(date)
projections_df.head()

,GameKey,PlayerID,SeasonType,Season,GameDate,Week,Team,Opponent,HomeOrAway,Number,...,DateTime,GlobalGameID,GlobalTeamID,GlobalOpponentID,ScoreID,FantasyPointsFantasyDraft,OffensiveFumbleRecoveryTouchdowns,SnapCountsConfirmed,Updated,ScoringDetails
0,202410104,19801,1,2024,2024-09-08T13:00:00,1,BUF,ARI,HOME,17,...,2024-09-08 13:00:00,18676,4,1,18676,33.7,NaN,NaN,2024-09-08T12:52:41,[]
1,202410126,21831,1,2024,2024-09-06T20:15:00,1,PHI,GB,HOME,1,...,2024-09-06 20:15:00,18683,26,12,18683,32.4,NaN,NaN,2024-09-08T12:52:41,[]
2,202410116,19781,1,2024,2024-09-05T20:20:00,1,BAL,KC,AWAY,8,...,2024-09-05 20:20:00,18684,3,16,18684,29.4,NaN,NaN,2024-09-08T12:52:41,[]
3,202410114,23239,1,2024,2024-09-08T13:00:00,1,HOU,IND,AWAY,7,...,2024-09-08 13:00:00,18679,13,14,18679,29.2,NaN,NaN,2024-09-08T12:52:41,[]
4,202410116,18890,1,2024,2024-09-05T20:20:00,1,KC,BAL,HOME,15,...,2024-09-05 20:20:00,18684,16,3,18684,29.1,NaN,NaN,2024-09-08T12:52:41,[]


In [64]:
def apply_projections(df_players, projections_df):
    df_players = df_players.merge(projections_df[['PlayerID', 'FantasyPointsDraftKings']], 
                        on='PlayerID', 
                        how='left')  # Merge on PlayerID

    # Replace ProjFantasyPoints with FantasyPointsDraftKings where available
    df_players['ProjFantasyPoints'] = df_players['FantasyPointsDraftKings'].combine_first(df_players['ProjFantasyPoints'])
            
    # Drop the temporary FantasyPointsDraftKings column
    df_players.drop(columns=['FantasyPointsDraftKings'], inplace=True)

    return df_players
    

In [66]:
def apply_stats(df_players, stats_df):
    df_players = df_players.merge(stats_df[['PlayerID', 'FantasyPointsDraftKings']], 
                        on='PlayerID', 
                        how='left')  # Merge on PlayerID

    # Replace ActualFantasyPoints with FantasyPointsDraftKings where available
    df_players['ActualFantasyPoints'] = df_players['FantasyPointsDraftKings'].combine_first(df_players['ActualFantasyPoints'])
            
    # Drop the temporary FantasyPointsDraftKings column
    df_players.drop(columns=['FantasyPointsDraftKings'], inplace=True)

    return df_players

In [70]:
df_players.head()

,SlatePlayerID,SlateID,SlateGameID,PlayerID,PlayerGameProjectionStatID,FantasyDefenseProjectionStatID,OperatorPlayerID,OperatorSlatePlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,Team,TeamID,RemovedByOperator,OperatorRosterSlots,ProjFantasyPoints,ActualFantasyPoints
0,2396406,12156,44096.0,21227,991161746.0,NaN,824428,35703473,Andrew Beck,RB,200,GB,12,False,[FLEX],0,0
1,2396407,12156,44096.0,21227,991161746.0,NaN,824428,35703478,Andrew Beck,RB,300,GB,12,False,[CPT],0,0
2,2396408,12156,44096.0,21797,991161770.0,NaN,922017,35703472,La'Mical Perine,RB,200,GB,12,False,[FLEX],0,0
3,2396409,12156,44096.0,21797,991161770.0,NaN,922017,35703477,La'Mical Perine,RB,300,GB,12,False,[CPT],0,0
4,2396410,12156,44096.0,24725,991161904.0,NaN,1049552,35703474,Griffin Hebert,WR,200,PHI,26,False,[FLEX],0,0


# Check if game_has_been_played has been initialized

In [72]:
df_players = apply_projections(df_players, projections_df)
if game_has_been_played:
    df_players = apply_stats(df_players, team1_stats_df)
    df_players = apply_stats(df_players, team2_stats_df)

In [74]:
df_players.head()

,SlatePlayerID,SlateID,SlateGameID,PlayerID,PlayerGameProjectionStatID,FantasyDefenseProjectionStatID,OperatorPlayerID,OperatorSlatePlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,Team,TeamID,RemovedByOperator,OperatorRosterSlots,ProjFantasyPoints,ActualFantasyPoints
0,2396406,12156,44096.0,21227,991161746.0,NaN,824428,35703473,Andrew Beck,RB,200,GB,12,False,[FLEX],0.0,0.0
1,2396407,12156,44096.0,21227,991161746.0,NaN,824428,35703478,Andrew Beck,RB,300,GB,12,False,[CPT],0.0,0.0
2,2396408,12156,44096.0,21797,991161770.0,NaN,922017,35703472,La'Mical Perine,RB,200,GB,12,False,[FLEX],0.0,0.0
3,2396409,12156,44096.0,21797,991161770.0,NaN,922017,35703477,La'Mical Perine,RB,300,GB,12,False,[CPT],0.0,0.0
4,2396410,12156,44096.0,24725,991161904.0,NaN,1049552,35703474,Griffin Hebert,WR,200,PHI,26,False,[FLEX],0.0,0.0


In [76]:
defenses_df = df_players.query("OperatorPosition == 'DST'")
defenses_df.head()

,SlatePlayerID,SlateID,SlateGameID,PlayerID,PlayerGameProjectionStatID,FantasyDefenseProjectionStatID,OperatorPlayerID,OperatorSlatePlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,Team,TeamID,RemovedByOperator,OperatorRosterSlots,ProjFantasyPoints,ActualFantasyPoints
50,2392931,12156,44096.0,12,NaN,990013600.0,335,35668011,Packers,DST,3600,GB,12,False,[FLEX],0.0,0.0
51,2392932,12156,44096.0,12,NaN,990013600.0,335,35668061,Packers,DST,5400,GB,12,False,[CPT],0.0,0.0
54,2392935,12156,44096.0,26,NaN,990013601.0,354,35668010,Eagles,DST,4000,PHI,26,False,[FLEX],0.0,0.0
55,2392936,12156,44096.0,26,NaN,990013601.0,354,35668060,Eagles,DST,6000,PHI,26,False,[CPT],0.0,0.0


In [78]:
defense_projections_df = read_defense_projections_json_file(date)
defense_projections_df.head()

,GameKey,SeasonType,Season,Week,Date,Team,Opponent,PointsAllowed,TouchdownsScored,SoloTackles,...,YahooTightEndFantasyPointsAllowed,YahooKickerFantasyPointsAllowed,FantasyPointsFantasyDraft,FantasyDraftFantasyPointsAllowed,FantasyDraftQuarterbackFantasyPointsAllowed,FantasyDraftRunningbackFantasyPointsAllowed,FantasyDraftWideReceiverFantasyPointsAllowed,FantasyDraftTightEndFantasyPointsAllowed,FantasyDraftKickerFantasyPointsAllowed,ScoringDetails
0,202410116,1,2024,1,2024-09-05 20:20:00,BAL,KC,17.6,0.1,0,...,NaN,NaN,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202410116,1,2024,1,2024-09-05 20:20:00,KC,BAL,15.8,0.1,0,...,NaN,NaN,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202410126,1,2024,1,2024-09-06 20:15:00,GB,PHI,18.6,0.1,0,...,NaN,NaN,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202410126,1,2024,1,2024-09-06 20:15:00,PHI,GB,17.4,0.1,0,...,NaN,NaN,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202410102,1,2024,1,2024-09-08 13:00:00,PIT,ATL,16.8,0.1,0,...,NaN,NaN,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
def apply_defense_projections(defenses_df, projections_df):
    defenses_df = defenses_df.merge(defense_projections_df[['Team', 'FantasyPointsDraftKings']], 
                        on='Team', 
                        how='left')  # Merge on PlayerID

    # Replace ProjFantasyPoints with FantasyPointsDraftKings where available
    defenses_df['ProjFantasyPoints'] = defenses_df['FantasyPointsDraftKings'].combine_first(defenses_df['ProjFantasyPoints'])
            
    # Drop the temporary FantasyPointsDraftKings column
    defenses_df.drop(columns=['FantasyPointsDraftKings'], inplace=True)

    return defenses_df

In [82]:
def apply_defense_projections_to_players(players_df, defenses_df):
    merged_df = players_df.merge(defenses_df[['SlatePlayerID', 'ProjFantasyPoints']], 
                                  on='SlatePlayerID', 
                                  how='left', 
                                  suffixes=('', '_defense'))
    
    # Update the ProjFantasyPoints in players_df with values from defenses_df
    players_df.loc[merged_df['ProjFantasyPoints_defense'].notnull(), 'ProjFantasyPoints'] = merged_df['ProjFantasyPoints_defense']

        
    return players_df

In [84]:
def apply_defense_stats(defenses_df, team_def_stats_df):
    defenses_df = defenses_df.merge(team_def_stats_df[['PlayerID', 'FantasyPointsDraftKings']], 
                        on='PlayerID', 
                        how='left')  # Merge on PlayerID

    # Replace ActualFantasyPoints with FantasyPointsDraftKings where available
    defenses_df['ActualFantasyPoints'] = defenses_df['FantasyPointsDraftKings'].combine_first(defenses_df['ActualFantasyPoints'])
            
    # Drop the temporary FantasyPointsDraftKings column
    defenses_df.drop(columns=['FantasyPointsDraftKings'], inplace=True)

    return defenses_df

In [86]:
defenses_df = apply_defense_projections(defenses_df, defense_projections_df)


In [88]:
defenses_df.head()

,SlatePlayerID,SlateID,SlateGameID,PlayerID,PlayerGameProjectionStatID,FantasyDefenseProjectionStatID,OperatorPlayerID,OperatorSlatePlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,Team,TeamID,RemovedByOperator,OperatorRosterSlots,ProjFantasyPoints,ActualFantasyPoints
0,2392931,12156,44096.0,12,NaN,990013600.0,335,35668011,Packers,DST,3600,GB,12,False,[FLEX],2.8,0.0
1,2392932,12156,44096.0,12,NaN,990013600.0,335,35668061,Packers,DST,5400,GB,12,False,[CPT],2.8,0.0
2,2392935,12156,44096.0,26,NaN,990013601.0,354,35668010,Eagles,DST,4000,PHI,26,False,[FLEX],3.2,0.0
3,2392936,12156,44096.0,26,NaN,990013601.0,354,35668060,Eagles,DST,6000,PHI,26,False,[CPT],3.2,0.0


In [90]:
df_players.head()

,SlatePlayerID,SlateID,SlateGameID,PlayerID,PlayerGameProjectionStatID,FantasyDefenseProjectionStatID,OperatorPlayerID,OperatorSlatePlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,Team,TeamID,RemovedByOperator,OperatorRosterSlots,ProjFantasyPoints,ActualFantasyPoints
0,2396406,12156,44096.0,21227,991161746.0,NaN,824428,35703473,Andrew Beck,RB,200,GB,12,False,[FLEX],0.0,0.0
1,2396407,12156,44096.0,21227,991161746.0,NaN,824428,35703478,Andrew Beck,RB,300,GB,12,False,[CPT],0.0,0.0
2,2396408,12156,44096.0,21797,991161770.0,NaN,922017,35703472,La'Mical Perine,RB,200,GB,12,False,[FLEX],0.0,0.0
3,2396409,12156,44096.0,21797,991161770.0,NaN,922017,35703477,La'Mical Perine,RB,300,GB,12,False,[CPT],0.0,0.0
4,2396410,12156,44096.0,24725,991161904.0,NaN,1049552,35703474,Griffin Hebert,WR,200,PHI,26,False,[FLEX],0.0,0.0


In [92]:

df_players_updated = apply_defense_projections_to_players(df_players, defenses_df)
if game_has_been_played:
    df_players_updated = apply_defense_stats(defenses_df, team1_def_stats_df)
    df_players_updated = apply_defense_stats(defenses_df, team2_def_stats_df)

In [98]:
df_players_updated.head(50)

,SlatePlayerID,SlateID,SlateGameID,PlayerID,PlayerGameProjectionStatID,FantasyDefenseProjectionStatID,OperatorPlayerID,OperatorSlatePlayerID,OperatorPlayerName,OperatorPosition,OperatorSalary,Team,TeamID,RemovedByOperator,OperatorRosterSlots,ProjFantasyPoints,ActualFantasyPoints
0,2392931,12156,44096.0,12,NaN,990013600.0,335,35668011,Packers,DST,3600,GB,12,False,[FLEX],2.8,0.0
1,2392932,12156,44096.0,12,NaN,990013600.0,335,35668061,Packers,DST,5400,GB,12,False,[CPT],2.8,0.0
2,2392935,12156,44096.0,26,NaN,990013601.0,354,35668010,Eagles,DST,4000,PHI,26,False,[FLEX],3.2,0.0
3,2392936,12156,44096.0,26,NaN,990013601.0,354,35668060,Eagles,DST,6000,PHI,26,False,[CPT],3.2,0.0


In [ ]:
df_players_updated['OperatorRosterSlots'] = df_players_updated['OperatorRosterSlots'].apply(lambda x: 'Captain' if x == ['CPT'] else 'Flex' if x == ['FLEX'] else x)


In [ ]:
df_players_updated.loc[df_players_updated['OperatorRosterSlots'] == 'Captain', 'ProjFantasyPoints'] *= 1.5
df_players_updated.loc[df_players_updated['OperatorRosterSlots'] == 'Captain', 'ActualFantasyPoints'] *= 1.5

In [ ]:
df_players_updated.head(40)

In [ ]:
def get_best_projected_lineup(df_norm):
    # Filter by captain and flex roles
    captain_df = df_norm[df_norm['OperatorRosterSlots'] == 'Captain'].copy()
    flex_df = df_norm[df_norm['OperatorRosterSlots'] == 'Flex'].copy()

    # Sort captains and flex players by salary in descending order
    captain_df = captain_df.sort_values(by='OperatorSalary', ascending=False)
    flex_df = flex_df.sort_values(by='OperatorSalary', ascending=False)
    
    # Initialize variables to track the best lineup
    best_lineup = None
    max_salary_used = 0
    best_fantasy_points = 0
    
    # Iterate through the sorted captains and try to build the best lineup
    for captain in captain_df.itertuples():
        # Start with the captain's salary and points
        total_salary = captain.OperatorSalary
        total_fantasy_points = captain.ProjFantasyPoints
        
        # Initialize a list for the flex lineup
        flex_lineup = []
        
        # Greedily select the top 5 flex players to maximize salary without exceeding the cap
        for flex in flex_df.itertuples():
            if total_salary + flex.OperatorSalary <= 50000:
                flex_lineup.append(flex)
                total_salary += flex.OperatorSalary
                total_fantasy_points += flex.ProjFantasyPoints
            
            # Stop once we have selected 5 flex players
            if len(flex_lineup) == 5:
                break
        
        # Check if this lineup uses more salary and has 5 flex players
        if len(flex_lineup) == 5 and total_salary > max_salary_used:
            max_salary_used = total_salary
            best_fantasy_points = total_fantasy_points
            best_lineup = [captain] + flex_lineup
    
    # Extract the names of the players in the best lineup
    if best_lineup:
        best_names = [player.OperatorPlayerName for player in best_lineup]
        return best_names, max_salary_used, best_fantasy_points
    else:
        return [], 0, 0

In [ ]:
def get_best_actual_lineup(df_norm):
    # Filter by captain and flex roles
    captain_df = df_norm[df_norm['OperatorRosterSlots'] == 'Captain'].copy()
    flex_df = df_norm[df_norm['OperatorRosterSlots'] == 'Flex'].copy()

    # Sort captains and flex players by salary in descending order
    captain_df = captain_df.sort_values(by='OperatorSalary', ascending=False)
    flex_df = flex_df.sort_values(by='OperatorSalary', ascending=False)
    
    # Initialize variables to track the best lineup
    best_lineup = None
    max_salary_used = 0
    best_fantasy_points = 0
    
    # Iterate through the sorted captains and try to build the best lineup
    for captain in captain_df.itertuples():
        # Start with the captain's salary and points
        total_salary = captain.OperatorSalary
        total_fantasy_points = captain.ActualFantasyPoints
        
        # Initialize a list for the flex lineup
        flex_lineup = []
        
        # Greedily select the top 5 flex players to maximize salary without exceeding the cap
        for flex in flex_df.itertuples():
            if total_salary + flex.OperatorSalary <= 50000:
                flex_lineup.append(flex)
                total_salary += flex.OperatorSalary
                total_fantasy_points += flex.ActualFantasyPoints
            
            # Stop once we have selected 5 flex players
            if len(flex_lineup) == 5:
                break
        
        # Check if this lineup uses more salary and has 5 flex players
        if len(flex_lineup) == 5 and total_salary > max_salary_used:
            max_salary_used = total_salary
            best_fantasy_points = total_fantasy_points
            best_lineup = [captain] + flex_lineup
    
    # Extract the names of the players in the best lineup
    if best_lineup:
        best_names = [player.OperatorPlayerName for player in best_lineup]
        return best_names, max_salary_used, best_fantasy_points
    else:
        return [], 0, 0

# Projected Lineup

In [ ]:
best_lineup, max_salary, max_points = get_best_projected_lineup(df_players_updated)
print(f"Best Lineup: {best_lineup}")
print(f"Total Salary Used: {max_salary}")
print(f"Total Projected Fantasy Points: {max_points}")

# Actual Lineup

In [ ]:
if game_has_been_played:
    best_lineup, max_salary, max_points = get_best_actual_lineup(df_players_updated)
    print(f"Best Lineup: {best_lineup}")
    print(f"Total Salary Used: {max_salary}")
    print(f"Total Actual Fantasy Points: {max_points}")

# Get Top 5 Lineups

In [ ]:
def get_top_5_projected_lineups(df_norm):
    # Filter by captain and flex roles
    captain_df = df_norm[df_norm['OperatorRosterSlots'] == 'Captain'].copy()
    flex_df = df_norm[df_norm['OperatorRosterSlots'] == 'Flex'].copy()

    # Sort captains and flex players by salary in descending order
    captain_df = captain_df.sort_values(by='OperatorSalary', ascending=False)
    flex_df = flex_df.sort_values(by='OperatorSalary', ascending=False)

    # Initialize a list to track the top lineups
    top_lineups = []

    # Iterate through the sorted captains and try to build lineups
    for captain in captain_df.itertuples():
        # Start with the captain's salary and points
        total_salary = captain.OperatorSalary
        total_fantasy_points = captain.ProjFantasyPoints

        # Initialize a list for the flex lineup
        flex_lineup = []

        # Exclude the captain from flex options to prevent duplicates
        available_flex_df = flex_df[flex_df['OperatorPlayerName'] != captain.OperatorPlayerName]

        # Greedily select flex players to maximize fantasy points without exceeding the cap
        for flex in available_flex_df.itertuples():
            # Check for duplicate players
            flex_player_names = [player.OperatorPlayerName for player in flex_lineup]
            if flex.OperatorPlayerName in flex_player_names:
                continue  # Skip if player is already selected

            if total_salary + flex.OperatorSalary <= 50000:
                flex_lineup.append(flex)
                total_salary += flex.OperatorSalary
                total_fantasy_points += flex.ProjFantasyPoints

            # Stop once we have selected 5 flex players
            if len(flex_lineup) == 5:
                break

        # Check if this lineup has 5 flex players and doesn't exceed the salary cap
        if len(flex_lineup) == 5:
            lineup = [captain] + flex_lineup
            top_lineups.append((lineup, total_salary, total_fantasy_points))

    # Sort the lineups by total fantasy points in descending order
    top_lineups = sorted(top_lineups, key=lambda x: x[2], reverse=True)

    # Ensure lineups are unique based on player names
    unique_lineups = []
    seen_lineups = set()
    for lineup_info in top_lineups:
        lineup, salary, fantasy_points = lineup_info
        player_names = tuple(sorted([player.OperatorPlayerName for player in lineup]))
        if player_names not in seen_lineups:
            seen_lineups.add(player_names)
            unique_lineups.append(lineup_info)
            if len(unique_lineups) == 5:
                break

    # Extract player names, salaries, and fantasy points for the top 5 unique lineups
    result = []
    for lineup, salary, fantasy_points in unique_lineups:
        player_names = []
        for idx, player in enumerate(lineup):
            if idx == 0:
                # First player is the captain
                player_names.append(f"{player.OperatorPlayerName} (Captain)")
            else:
                player_names.append(player.OperatorPlayerName)
        result.append((player_names, salary, fantasy_points))

    return result

In [ ]:
def get_top_5_actual_lineups(df_norm):
    # Filter by captain and flex roles
    captain_df = df_norm[df_norm['OperatorRosterSlots'] == 'Captain'].copy()
    flex_df = df_norm[df_norm['OperatorRosterSlots'] == 'Flex'].copy()

    # Sort captains and flex players by salary in descending order
    captain_df = captain_df.sort_values(by='OperatorSalary', ascending=False)
    flex_df = flex_df.sort_values(by='OperatorSalary', ascending=False)

    # Initialize a list to track the top lineups
    top_lineups = []

    # Iterate through the sorted captains and try to build lineups
    for captain in captain_df.itertuples():
        # Start with the captain's salary and points
        total_salary = captain.OperatorSalary
        total_fantasy_points = captain.ActualFantasyPoints

        # Initialize a list for the flex lineup
        flex_lineup = []

        # Exclude the captain from flex options to prevent duplicates
        available_flex_df = flex_df[flex_df['OperatorPlayerName'] != captain.OperatorPlayerName]

        # Greedily select flex players to maximize fantasy points without exceeding the cap
        for flex in available_flex_df.itertuples():
            # Check for duplicate players
            flex_player_names = [player.OperatorPlayerName for player in flex_lineup]
            if flex.OperatorPlayerName in flex_player_names:
                continue  # Skip if player is already selected

            if total_salary + flex.OperatorSalary <= 50000:
                flex_lineup.append(flex)
                total_salary += flex.OperatorSalary
                total_fantasy_points += flex.ActualFantasyPoints

            # Stop once we have selected 5 flex players
            if len(flex_lineup) == 5:
                break

        # Check if this lineup has 5 flex players and doesn't exceed the salary cap
        if len(flex_lineup) == 5:
            lineup = [captain] + flex_lineup
            top_lineups.append((lineup, total_salary, total_fantasy_points))

    # Sort the lineups by total fantasy points in descending order
    top_lineups = sorted(top_lineups, key=lambda x: x[2], reverse=True)

    # Ensure lineups are unique based on player names
    unique_lineups = []
    seen_lineups = set()
    for lineup_info in top_lineups:
        lineup, salary, fantasy_points = lineup_info
        player_names = tuple(sorted([player.OperatorPlayerName for player in lineup]))
        if player_names not in seen_lineups:
            seen_lineups.add(player_names)
            unique_lineups.append(lineup_info)
            if len(unique_lineups) == 5:
                break

    # Extract player names, salaries, and fantasy points for the top 5 unique lineups
    result = []
    for lineup, salary, fantasy_points in unique_lineups:
        player_names = []
        for idx, player in enumerate(lineup):
            if idx == 0:
                # First player is the captain
                player_names.append(f"{player.OperatorPlayerName} (Captain)")
            else:
                player_names.append(player.OperatorPlayerName)
        result.append((player_names, salary, fantasy_points))

    return result

# Top 5 projections

In [ ]:
# Assuming df_players_updated is your DataFrame
top_lineups = get_top_5_projected_lineups(df_players_updated)

# Print out the results
for i, (player_names, salary, fantasy_points) in enumerate(top_lineups, 1):
    print(f"Lineup {i}:")
    print(f"Players: {', '.join(player_names)}")
    print(f"Total Salary: ${salary}")
    print(f"Total Projected Fantasy Points: {fantasy_points}")
    print('-' * 30)

# Top 5 Actual

In [ ]:
if game_has_been_played:
    top_lineups = get_top_5_actual_lineups(df_players_updated)
    
    # Print out the results
    for i, (player_names, salary, fantasy_points) in enumerate(top_lineups, 1):
        print(f"Lineup {i}:")
        print(f"Players: {', '.join(player_names)}")
        print(f"Total Salary: ${salary}")
        print(f"Total Actual Fantasy Points: {fantasy_points}")
        print('-' * 30)